In [1]:
!pip install facenet_pytorch
import torch
import torch.nn.functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
from PIL import Image
import warnings
warnings.filterwarnings("ignore")


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

mtcnn = MTCNN(
    select_largest=False,
    post_process=False,
    device=DEVICE
).to(DEVICE).eval()

In [3]:
model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
)

checkpoint = torch.load("resnetinceptionv1_epoch_32.pth", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
model.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [4]:
def predict(image_path:str):
    
    input_image = Image.open(image_path).convert('RGB')
    face = mtcnn(input_image)
    if face is None:
        raise Exception('No face detected')
    face = face.unsqueeze(0)
    face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)
    
    prev_face = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()
    prev_face = prev_face.astype('uint8')

    face = face.to(DEVICE)
    face = face.to(torch.float32)
    face = face / 255.0
    face_image_to_plot = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()

    target_layers=[model.block8.branch1[-1]]
    use_cuda = True if torch.cuda.is_available() else False
    

    with torch.no_grad():
        output = torch.sigmoid(model(face).squeeze(0))
        #print(output)
        prediction = "real" if output.item() < 0.5 else "fake"
        
        real_prediction = 1 - output.item()
        fake_prediction = output.item()
        
        confidences = {
            'real': real_prediction,
            'fake': fake_prediction
        }
    return confidences, prediction



predict(r"C:\Users\hp\python_jupyter\py_dsa\data\fake\fake_frame_2.png")



({'real': 0.00011217594146728516, 'fake': 0.9998878240585327}, 'fake')

In [ ]:
# Accuracy
import os
import torch
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F

def predict(input_path: str):
    model.eval()

    input_image = Image.open(input_path).convert('RGB')
    face = mtcnn(input_image)
    
    if face is None:
        raise Exception('No face detected')
    
    face = face.unsqueeze(0)
    face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)

    face = face.to(device)
    face = face.to(torch.float32)
    face = face / 255.0

    with torch.no_grad():
        output = torch.sigmoid(model(face).squeeze(0))
        
        prediction = "real" if output.item() < 0.5 else "fake"
        real_prediction = 1 - output.item()
        fake_prediction = output.item()

        confidences = {
            'real': real_prediction,
            'fake': fake_prediction
        }

    return prediction, confidences

def calculate_accuracy(model, mtcnn, device, dataset_path):
    correct_predictions = 0
    total_predictions = 0
    
    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)
        
        if os.path.isdir(label_path):
            for file in os.listdir(label_path):
                if file.endswith(".jpg") or file.endswith(".png"):
                    total_predictions += 1
                    try:
                        image_path = os.path.join(label_path, file)
                        
                        predicted_label, _ = predict(image_path)
                        
                        print(f"Image: {file}, Predicted: {predicted_label}, Actual: {label}")
                        
                        if predicted_label == label:
                            correct_predictions += 1
                    
                    except Exception as e:
                        print(f"Error processing {image_path}: {e}")
    
    accuracy = (correct_predictions / total_predictions) * 100
    print(f"\nTotal images: {total_predictions}")
    print(f"Correct predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.2f}%")

if __name__== "__main__":
    # Assuming you have already loaded the model, mtcnn, and set the device
   
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset_path = r"C:\Users\hp\python_jupyter\py_dsa\data"  
    calculate_accuracy(model, mtcnn, device, dataset_path)